In [ ]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
import nltk
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
import inflect

In [ ]:
# English inflection helper (used to detect singular/plural of input terms)
p = inflect.engine()

In [ ]:
ITALIAN_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Tutti i {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Senza eccezione, tutti i {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ogni {a_sg} è {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Tutto ciò che è {a_sg_art} è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Ognuno dei {a_pl} rientra in {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Se qualcosa è {a_sg_art}, allora è certamente {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'La classe dei {a_pl} è inclusa in quella dei {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Qualsiasi {a_sg} si può definire come {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Non esiste alcun {a_sg} che non sia {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Essere {a_sg_art} implica necessariamente essere {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'La totalità dei {a_pl} è composta da {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'È un dato di fatto che ogni {a_sg} sia {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Ciascun membro di {a_pl} appartiene anche a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Qualunque {a_sg} rientra nella categoria di {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Gli {a_pl} sono universalmente considerati {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Alcuni {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Esiste almeno un {a_sg} che è {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Ci sono {a_pl} che sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Almeno alcuni {a_pl} risultano essere {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Un ristretto numero di {a_pl} è {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Certi {a_pl} sono anche {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si danno casi in cui {a_sg_art} è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'È possibile trovare dei {a_pl} che appartengono a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Almeno un esempio di {a_sg} è identificato come {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Una parte della classe {a_pl} è {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Non tutti, ma alcuni {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Tra i {a_pl}, se ne trovano diversi che sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si può affermare che qualche {a_sg} sia {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Sussistono {a_pl} che presentano le qualità di {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Capita che alcuni {a_pl} siano {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Nessun {a_pl} è {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Non esiste alcun {a_sg} che sia {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Nulla che sia {a_sg_art} è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Non ci sono {a_pl} che siano {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Mai i {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nessuno dei {a_pl} appartiene alla classe dei {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'È falso che un {a_sg} possa essere {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'La classe {a_pl} è disgiunta dalla classe {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Assolutamente nessun {a_sg} rientra in {b_pl}.', 'a_type': 'singular', 'b_type': 'plural'},
            {'tpl': 'In nessun caso un {a_sg} è {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'È escluso che i {a_pl} siano {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Se è {a_sg_art}, non può in alcun modo essere {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Non è possibile rintracciare un {a_sg} che sia {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Ogni membro di {a_pl} è diverso da {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Lintersezione tra {a_pl} e {b_pl} è nulla.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Alcuni {a_pl} non sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Non ogni {a_sg} è {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Alcuni {a_pl} non appartengono a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Non tutti i {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ci sono {a_pl} che non sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Certi {a_pl} sono privi della qualità di {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Esiste almeno un {a_sg} che non è {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Non si può dire che ogni membro di {a_pl} sia {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Almeno un caso di {a_sg_art} non è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'È falso che la totalità dei {a_pl} sia {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Una parte dei {a_pl} resta fuori dalla categoria {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Si possono trovare {a_pl} che non risultano essere {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Non tutto ciò che è {a_sg_art} è anche {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Alcuni elementi di {a_pl} non sono membri di {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Esistono esemplari di {a_pl} che non sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ]
    },
    'conclusions': [
        'Pertanto, {conclusion}.',
        'Quindi, {conclusion}.',
        'Di conseguenza, {conclusion}.',
        'Da ciò si può dedurre che {conclusion}.',
        'Ne consegue che {conclusion}.',
        'Si conclude che {conclusion}.',
        'Risulta chiaro che {conclusion}.',
        'In conclusione, {conclusion}.',
        'Da questo deriva che {conclusion}.',
        'Per tale ragione, {conclusion}.'
    ],
}

In [ ]:
ITALIAN_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Ogni {d_sg} che è {a_sg_art} è anche {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Se {d_sg_art} è {a_sg_art}, allora deve essere {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Tutti i {d_pl} che sono {a_pl} sono anche {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Senza eccezione, i {d_pl} che sono {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ogni {d_sg} appartenente a {a_sg_art} risulta essere {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Qualunque {d_sg} che si qualifichi come {a_sg_art} è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Ogni oggetto che sia sia {d_sg_art} che {a_sg_art} è anche {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'La totalità dei {d_pl} che sono {a_pl} sono contemporaneamente {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Allinterno di {d_pl}, tutti i {a_pl} appartengono a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Per ogni {d_sg}, se è {a_sg_art} allora è necessariamente {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Tutto ciò che è simultaneamente {d_sg_art} e {a_sg_art} è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Ogni membro di {d_pl} che appartiene a {a_pl} fa parte di {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ciascuno dei {d_pl} che è {a_pl} rientra in {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Non esiste alcun {d_sg} che, essendo {a_sg_art}, non sia {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Ogni istanza di {d_sg} che possiede la proprietà {a_sg} ha anche {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'some': [
            {'tpl': 'Esiste almeno un {d_sg} che è sia {a_sg_art} che {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Alcuni {d_pl} sono sia {a_pl} che {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ci sono {d_pl} che sono {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Almeno uno dei {d_pl} è {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Un {d_sg} esiste che è {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Certi {d_pl} risultano essere contemporaneamente {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'È possibile individuare un {d_sg} che sia {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Capita che qualche {d_sg} sia allo stesso tempo {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Una porzione di {d_pl} presenta le caratteristiche di {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Tra i {d_pl}, si trova almeno un esempio che è sia {a_sg} che {b_sg}.', 'a_type': 'plural', 'b_type': 'singular', 'd_type': 'plural'},
            {'tpl': 'Possiamo rintracciare dei {d_pl} che soddisfano sia {a_pl} che {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Qualche {d_sg} è a un tempo {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Allinterno della categoria {d_pl}, esistono {a_pl} che sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Diversi {d_pl} condividono le proprietà di {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Alcuni elementi di {d_pl} sono identificati come {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Nessuno dei {d_pl} che sono {a_pl} è {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Non ci sono {d_pl} che siano {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Mai i {d_pl} che sono {a_pl} sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Non esiste {d_sg} che sia {a_sg} e {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Nessun {d_sg} che sia {a_sg} è {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'È impossibile che un {d_sg} sia sia {a_sg_art} che {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Nessun {d_sg} con la qualità {a_sg} possiede anche {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Non si trova alcun {d_sg} che sia a un tempo {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Allinterno di {d_pl}, nessun {a_pl} appartiene a {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Fica escluso che i {d_pl} che siano {a_pl} siano anche {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'È falso che qualche {d_sg} possa essere simultaneamente {a_sg_art} e {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Nessun membro di {d_pl} appartiene a {a_pl} e {b_pl} allo stesso tempo.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Se è {d_sg_art} e {a_sg_art}, allora non può assolutamente essere {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Non cè modo che un {d_sg} sia sia {a_sg} che {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'La categoria {d_pl} non ammette la coesistenza di {a_pl} e {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Non ogni {d_sg} che è {a_sg_art} è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Ci sono {d_pl} che sono {a_pl} ma non {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Alcuni {d_pl} che sono {a_pl} non sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Almeno uno dei {d_pl} che è {a_pl} non è {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Un {d_sg} che sia {a_sg} potrebbe non essere {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Certi {d_pl} appartenenti a {a_pl} mancano della proprietà {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Esiste almeno un {d_sg} che, pur essendo {a_sg_art}, non è {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Non è vero che ogni {d_sg} che sia {a_sg_art} debba essere anche {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Si possono individuare dei {d_pl} che sono {a_pl} ma restano fuori da {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Allinterno della classe {d_pl}, alcuni {a_pl} non sono {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Non tutto ciò che è contemporaneamente {d_sg_art} e {a_sg_art} risulta essere {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Qualche {d_sg} che è {a_sg} non è {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'È possibile che un {d_sg} sia {a_sg} e tuttavia non sia {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Diversi membri di {d_pl} che sono {a_pl} non sono membri di {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Una parte di {d_pl} è {a_pl} ma non è {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ]
    }
}

In [ ]:
def get_singular_and_plural(term):
    """Return (singular, plural) using English inflect heuristics (fallback to term).
    """
    term = term.strip()
    try:
        sg_attempt = p.singular_noun(term)
    except Exception:
        sg_attempt = False
    if sg_attempt is not False and sg_attempt is not None:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        try:
            pl = p.plural_noun(term) or term
        except Exception:
            pl = term
    return sg, pl

In [ ]:
def get_article_italian(term):
    """Simple Italian indefinite article heuristic: 'una' if word ends with 'a', else 'un'.
    This is heuristic-only and won't fully handle gender exceptions.
    """
    term = term.strip()
    if term.endswith('a'):
        return 'una ' + term
    return 'un ' + term

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    match = re.match(r'All (.*) that are (.*) are (.*).', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*).', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*).', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*).', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    match = re.match(r'All (.*) are (.*).', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*).', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*).', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*).', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse basic syllogism type from sentence: {sentence}')

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth, synonym_dict=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article_italian(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article_italian(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article_italian(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if isinstance(val, str) and s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    premises_simple, conclusion_simple = split_syllogism_into_sentences(simple_syllogism)
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def split_syllogism_into_sentences(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    if len(sentences) < 2:
        raise ValueError(f'Syllogism must contain at least one premise and one conclusion: {syllogism}')
    sentences = [s + '.' for s in sentences]
    conclusion = sentences.pop()
    premises = sentences
    return premises, conclusion

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f' Skipping item {item.get(id, 'N/A')} due to parse error: {e}')
            continue
    return complex_dataset

In [ ]:
# Example English input data
sample_data = [
    {
        'id': 'it-1',
        'syllogism': 'All dogs are animals. All animals are living things. Some dogs are living things.',
        'validity': True,
        'plausibility': True
    },
    {
        'id': 'it-2',
        'syllogism': 'No fish are mammals. Some mammals are whales. Some whales are not fish.',
        'validity': True,
        'plausibility': True
    }
]
# Run conversion using Italian templates
complex_syllogism_data = process_dataset(sample_data, ITALIAN_COMPLEX_PATTERNS, ITALIAN_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))